In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/time-track/time_track.csv


In [2]:
data = pd.read_csv("/kaggle/input/time-track/time_track.csv")

#data.loc[57, 'How many clients do you usually (would you) take on at a time?']
#data.loc[57, 'How many clients do you usually (would you) take on at a time?']
# recup ligne qu'on va enlever aux donnees finales

In [3]:
#average number of missing value in columns
L = []
for col in data.columns:
    L.append(data[col].isnull().sum())
n = len(L)
s=0
for i in L:
    s += i
print(s/n)


30.145631067961165


In [4]:
cols_with_lot_missing = [ col for col in data.columns
                        if (data[col].isnull().sum() > 30)]

len(cols_with_lot_missing) #in 103 cols

52

In [5]:
#average number of question people don't answer
L = []
for row in data.index:
    L.append(data.iloc[row, :].isnull().sum())
n = len(L)
s=0
for i in L:
    s += i
print(s/n)

35.28409090909091


In [6]:
rows_with_lot_missing = [row for row in data.index
                        if data.iloc[row, :].isnull().sum() > 35]

len(rows_with_lot_missing) #in 88 rows

36

In [7]:
data2 = data.drop(cols_with_lot_missing, axis = 1)
data3 = data2.drop(rows_with_lot_missing, axis = 0)

print(data3.shape)
#data3

(52, 51)


In [8]:
#distinction num varialble and cat variables
num_col = [col for col in data3.columns
          if data3[col].dtype in ['int64', 'float64']]
len(num_col)

38

In [9]:
object_col = [col for col in data3.columns
             if data3[col].dtype == 'object']
len(object_col)

13

In [10]:
#average unique value for each col in object_col
n = len(object_col)
s = 0

for col in object_col[1:]:
    s += data3[col].unique().shape[0]
    
print(s/n)


25.846153846153847


In [11]:
#cleaning data I'm not going to work with

#1 droping col because of their unicity

cat_col_to_remove = [col for col in object_col
          if data3[col].unique().shape[0] > 25] #10 is advised but with 10 or 25 we get the same result

cat_col_to_remove 

['#',
 "Lastly, would you mind sharing the name of a few companies you've worked with as a NetSuite professional?",
 "Any preferred industry you'd like to work in?",
 'We would love to follow up with you. Please leave us your email.',
 'Please enter your first and last name.',
 'Start Date (UTC)',
 'Submit Date (UTC)',
 'Network ID']

In [12]:
data4 = data3.drop(cat_col_to_remove, axis = 1)
#data4

In [13]:
#2 droping row because of their missing value in the col we want to predict

to_predict = 'How many clients do you usually (would you) take on at a time?'
row_to_remove = []

for i in data4[to_predict].index:
    if type(data4.loc[i, to_predict]) != str: # if not string it is NaN
        row_to_remove.append(i) 
        
row_to_remove

[4, 18, 21, 26, 46, 54, 58, 59, 64, 66, 68, 85]

In [14]:
data5 = data4.drop(row_to_remove, axis = 0)
data5.shape # (40, 43) -> looks good

(40, 43)

In [15]:
# method : simple imputer for numerical values and hot encoder for categorical ones

In [16]:
#distinction num varialble and cat variables
num_cole = [col for col in data5.columns
          if data5[col].dtype in ['int64', 'float64']]
len(num_col)

38

In [17]:
#distinction num varialble and cat variables removing the col we want to predict
object_cole = [col for col in data5.columns
          if ((data5[col].dtype == 'object') & (col != ('How many clients do you usually (would you) take on at a time?'))) ]
len(object_cole)

4

In [18]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

X = data5.drop(['How many clients do you usually (would you) take on at a time?'], axis = 1)
y_bis = data5['How many clients do you usually (would you) take on at a time?']

# I am taking the average number given ( 2 or 3 become 2.5)
y = y_bis.copy()

for i in y.index:
    if len(y[i])>2:
        #print(len(y[i])+y[i][0])
        y[i] = (float(y[i][0])+float(y[i][len(y[i])-1]))/2
    else:
        y[i] = y[i][0] 

X_train, X_val, y_train, y_val = train_test_split(X, y)
#X.iloc[13,:]

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [19]:
'''# dealing with categorial variables
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

my_encoder = OneHotEncoder(handle_unknown = 'ignore', sparse = False) #handle_unknown = 'ignore' lets me deal with dates
my_imputer = SimpleImputer(strategy = 'most_frequent')
#cannot do that
#encoded_X_train = X_train.copy()
#encoded_X_valid = X_val.copy()

encoded_X_train = pd.DataFrame(my_encoder.fit_transform(X_train[object_cole]))
encoded_X_val = pd.DataFrame(my_encoder.transform(X_val[object_cole]))

imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train[num_cole]))
imputed_X_val = pd.DataFrame(my_imputer.fit_transform(X_val[num_cole]))


#put back index for encoding
encoded_X_train.index = X_train.index
encoded_X_val.index = X_val.index
#print(encoded_X_train)

#put back col for imputing
imputed_X_train.columns = X_train[num_cole].columns
imputed_X_val.columns = X_val[num_cole].columns
#print(imputed_X_train)
#remove cat col, we'll replace them by the encoded ones
#num_X_train = X_train.drop(object_cole, axis = 1)
#num_X_val = X_val.drop(object_cole, axis = 1)

#add one hot encoded to numerical features
X_train_f = pd.concat([encoded_X_train, imputed_X_train], axis = 1)
#print(X_train_f)
X_val_f = pd.concat([encoded_X_val, imputed_X_val], axis = 1)
#print(X_train_f)

#check that all the values have the same type (str)
X_train_f.columns = X_train_f.columns.astype(str)
X_val_f.columns = X_val_f.columns.astype(str)

#OHE only remove index, following lines are unnecessary
#encoded_X_train.columns = X_train.columns
#encoded_X_valid.columns = X_valid.columns

#some coded data don't have info in categorical columns ans vice versa, I have to use pipelines'''



"# dealing with categorial variables\nfrom sklearn.preprocessing import OneHotEncoder\nfrom sklearn.impute import SimpleImputer\n\nmy_encoder = OneHotEncoder(handle_unknown = 'ignore', sparse = False) #handle_unknown = 'ignore' lets me deal with dates\nmy_imputer = SimpleImputer(strategy = 'most_frequent')\n#cannot do that\n#encoded_X_train = X_train.copy()\n#encoded_X_valid = X_val.copy()\n\nencoded_X_train = pd.DataFrame(my_encoder.fit_transform(X_train[object_cole]))\nencoded_X_val = pd.DataFrame(my_encoder.transform(X_val[object_cole]))\n\nimputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train[num_cole]))\nimputed_X_val = pd.DataFrame(my_imputer.fit_transform(X_val[num_cole]))\n\n\n#put back index for encoding\nencoded_X_train.index = X_train.index\nencoded_X_val.index = X_val.index\n#print(encoded_X_train)\n\n#put back col for imputing\nimputed_X_train.columns = X_train[num_cole].columns\nimputed_X_val.columns = X_val[num_cole].columns\n#print(imputed_X_train)\n#remove ca

In [20]:
'''#dealing with numerical variables
my_imputer = SimpleImputer(strategy = "most_frequent" )# I think it would be the most accurate(if they didn't understand the question) or maybe 0 (if they haven't any skill on that domain)

imput_encod_X_train = encoded_X_train_f.copy()
imput_encod_X_val = encoded_X_val_f.copy()

imput_encod_X_train[num_cole] = pd.DataFrame(my_imputer.fit_transform(encoded_X_train_f[num_cole]))
imput_encod_X_val[num_cole] = pd.DataFrame(my_imputer.transform(encoded_X_val_f[num_cole]))

#imputer remove col, we put them back
imput_encod_X_train.columns = encoded_X_train_f.columns
imput_encod_X_val.columns = encoded_X_val_f.columns

#no longer necessary, I did both imputing en encoding in the cell above
#some coded data don't have info in categorical columns ans vice versa, I have to use pipelines'''

'#dealing with numerical variables\nmy_imputer = SimpleImputer(strategy = "most_frequent" )# I think it would be the most accurate(if they didn\'t understand the question) or maybe 0 (if they haven\'t any skill on that domain)\n\nimput_encod_X_train = encoded_X_train_f.copy()\nimput_encod_X_val = encoded_X_val_f.copy()\n\nimput_encod_X_train[num_cole] = pd.DataFrame(my_imputer.fit_transform(encoded_X_train_f[num_cole]))\nimput_encod_X_val[num_cole] = pd.DataFrame(my_imputer.transform(encoded_X_val_f[num_cole]))\n\n#imputer remove col, we put them back\nimput_encod_X_train.columns = encoded_X_train_f.columns\nimput_encod_X_val.columns = encoded_X_val_f.columns\n\n#no longer necessary, I did both imputing en encoding in the cell above\n#some coded data don\'t have info in categorical columns ans vice versa, I have to use pipelines'

In [21]:
#X_train_f

In [22]:
#use of pipelines
# 1 defining preprocessing steps

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

#preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy = 'constant', fill_value = 0)

#preprocessing for categorical data
categorical_transformer = Pipeline( steps = [
    ('imputer', SimpleImputer(strategy = 'constant', fill_value = '0')),
    ('encoder', OneHotEncoder(handle_unknown = 'ignore'))
])

#gathering num and cat col by the preprocess
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, num_cole),
        ('cat', categorical_transformer, object_cole)
    ])


In [23]:
# 2 define a model

from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators = 500, random_state = 1)

# 3 creating and evaluating pipelines

from sklearn.metrics import mean_absolute_error

#gathering preprocessing and model code in a pipeline

my_pipeline = Pipeline( steps = [('preprocessor', preprocessor),
                                ('model', model)
                                ])

my_pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  SimpleImputer(fill_value=0,
                                                                strategy='constant'),
                                                  ['Sales activities / Biz Dev',
                                                   'NetSuite Solution '
                                                   'Architect',
                                                   'NetSuite configuration / '
                                                   'customization',
                                                   'Technical/ NetSuite '
                                                   'development / Systems '
                                                   'Integration',
                                                   'NetSuite Implementation '
                                                   'projects',
                                                   'Ongoing long-term (post '
                                                   'go-live) NetSuite '
                                                   'opt...
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='0',
                                                                                 strategy='constant')),
                                                                  ('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['How do you prefer to get '
                                                   'paid for your NetSuite '
                                                   'service?',
                                                   'What is your favorite type '
                                                   'of engagement?',
                                                   'How many implementation '
                                                   'projects have you been '
                                                   'part of?',
                                                   'What comes first?'])])),
                ('model',
                 RandomForestRegressor(n_estimators=500, random_state=1))])

In [34]:
X_val.head()

,Sales activities / Biz Dev,NetSuite Solution Architect,NetSuite configuration / customization,Technical/ NetSuite development / Systems Integration,NetSuite Implementation projects,Ongoing long-term (post go-live) NetSuite optimization projects,Ongoing long-term NetSuite admin and end-user support,A NetSuite optimization project occasionally,Are you interested in writing and monetizing NetSuite articles and tutorials?,How do you prefer to get paid for your NetSuite service?,...,How would you rate yourself in the* Software* vertical of NetSuite?,How would you rate yourself in the* Professional Services* vertical of NetSuite?,How would you rate yourself in the* Ecommerce* vertical of NetSuite?,How would you rate yourself in the* Health and Beauty* vertical of NetSuite?,How would you rate yourself in the* Financial Services* vertical of NetSuite?,How would you rate yourself in the* Media and Publishing* vertical of NetSuite?,"Right now, do you have bandwidth for more clients?",How is work life balance these days?,What comes first?,"Finally, would you be interested in freelance opportunities with OdeCloud?"
2,2.0,4.0,3.0,3.0,2.0,5.0,5.0,2.0,1.0,Fixed-price per project,...,7.0,2.0,1.0,2.0,3.0,2.0,0.0,2.0,Maximum cash (at least for now),1.0
80,3.0,2.0,5.0,5.0,5.0,3.0,5.0,2.0,1.0,Monthly retainer,...,5.0,5.0,3.0,7.0,6.0,7.0,0.0,5.0,Maximum cash (at least for now),1.0
76,2.0,2.0,5.0,5.0,5.0,5.0,5.0,2.0,1.0,Monthly retainer,...,6.0,5.0,6.0,5.0,6.0,7.0,0.0,5.0,Maximum cash (at least for now),1.0
53,3.0,4.0,3.0,1.0,5.0,4.0,5.0,3.0,1.0,Hourly,...,8.0,5.0,5.0,3.0,5.0,3.0,1.0,3.0,Maximum cash (at least for now),1.0
19,2.0,5.0,5.0,4.0,5.0,5.0,5.0,5.0,1.0,Monthly retainer,...,1.0,3.0,10.0,2.0,6.0,1.0,1.0,5.0,Maximum cash (at least for now),1.0


In [25]:
y_val

2     2.5
80    2.5
76    2.5
53    2.5
19    4.5
48    4.5
56    2.5
0     2.5
74      5
32    2.5
Name: How many clients do you usually (would you) take on at a time?, dtype: object

In [36]:
#un exemple

preds_test = my_pipeline.predict(X_val.iloc[:3, :]) # pas mal

#for i in range (len(preds_test)):
   # preds_test[i] = round(preds_test[i])


print(preds_test)

print(list(y_val)[:3]) #len(y_val) = 10

mean_absolute_error(preds_test, y_val[:3])

# focus on 6 first questions !!!

[3.148 3.042 3.34 ]
[2.5, 2.5, 2.5]


0.6766666666666666

In [37]:
# for all consultants

preds = my_pipeline.predict(X_val)

for i in range (len(preds)):
    preds[i] = round(preds[i])

#print(preds)

mean_absolute_error(preds, y_val)


0.75

In [28]:
#let's check the accuracy with cross validation :

from sklearn.model_selection import cross_val_score

scores = -1 * cross_val_score(my_pipeline, X, y,
                              cv = 5,
                              scoring = 'neg_mean_absolute_error')
scores.mean()

0.957075

In [29]:
# not satisfied + we don't have so much data => I'm going to use XGBoost for more accuracy

In [30]:
preprocessor.fit_transform(X_train)

array([[1., 4., 4., ..., 0., 0., 1.],
       [4., 5., 4., ..., 0., 0., 1.],
       [3., 5., 5., ..., 0., 0., 1.],
       ...,
       [2., 3., 5., ..., 0., 0., 1.],
       [2., 3., 5., ..., 1., 1., 0.],
       [3., 4., 5., ..., 0., 1., 0.]])

In [31]:
# a more complex model

from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error

model = XGBRegressor(n_estimators = 10000, learning_rate = 0.001, random_state = 1)

pipeline = Pipeline( steps = [('preprocessor', preprocessor),
                                ('model', model)
                                ])

preprocessor.fit_transform(X_train)
#print(X_train.iloc[1,:])
X_val_transformed = preprocessor.transform(X_val)

fit_params = {'model__eval_set' : [(X_val_transformed, y_val)],
             'model__early_stopping_rounds' : 5,
             'model__verbose' : False}

pipeline.fit(X_train, y_train, **fit_params)

preds = pipeline.predict(X_val)

mean_absolute_error(preds, y_val)

/opt/conda/lib/python3.10/site-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


0.7565183639526367

In [32]:
#un exemple

preds_test = pipeline.predict(X_val.iloc[:2, :]) # pas mal

for i in range (len(preds_test)):
    preds_test[i] = round(preds_test[i])


print(preds_test)
print(list(y_val)[:2])

mean_absolute_error(preds_test, y_val[:2])

[3. 2.]
[2.5, 2.5]


0.5

In [33]:
# to reconsider
''''my_pipeline.fit(X_train, y_train)
early_stopping_rounds = 5)
preds = my_pipeline.predict(X_val)
mean_absolute_error(preds, y_val)'''

"'my_pipeline.fit(X_train, y_train)\nearly_stopping_rounds = 5)\npreds = my_pipeline.predict(X_val)\nmean_absolute_error(preds, y_val)"